In [1]:
import os
import copy
import numpy as np
import pathlib
import rasterio
import geopandas as gpd
import pandas as pd

from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook

from food_security import data_reader
from rasterio.transform import from_origin
from rasterio.mask import mask
import xarray as xr

In [2]:
src_dir = pathlib.Path('~').expanduser().resolve() / 'data/food-security'
# src_dir = pathlib.Path('/Users/hemert/OneDrive - Stichting Deltares/Documents - International Delta Toolset/Agr production postprocessing/JCARWQV7.Rbd/1/')
ribasim_dir = src_dir / 'Ribasim8_MekDelta/Modules/ribasim/Mekong.1/work/'

In [3]:
his_file = data_reader.HisFile(ribasim_dir / 'RIB_ADVIR_dmnd.his')
his_file = data_reader.HisFile(src_dir / 'AirAgDmd.his')
his_file.read()
ds = his_file.ds

In [4]:
ds

<xarray.Dataset> Size: 2MB
Dimensions:                 (time: 72, station: 68)
Coordinates:
  * time                    (time) datetime64[ns] 576B 2020-10-01 ... 2022-09-21
  * station                 (station) <U20 5kB '5 / Blk_Air_20' ... '1147 / B...
Data variables: (12/97)
    Field crop water req    (time, station) float32 20kB 6.331 6.609 ... 6.775
    Field crop water req_2  (time, station) float32 20kB 0.7327 ... 0.7842
    Demand if feedback (    (time, station) float32 20kB 8.984 9.379 ... 9.615
    Demand if feedback (_2  (time, station) float32 20kB 1.04 1.086 ... 1.113
    Supply (mm/day)         (time, station) float32 20kB 8.984 9.379 ... 2.12
    Supply (l/s/ha)         (time, station) float32 20kB 1.04 1.086 ... 0.2454
    ...                      ...
    A Cr29/Sesame_SESAME    (time, station) float32 20kB 0.0 0.0 0.0 ... 0.0 0.0
    A Cr30/Peanut_GNUT1     (time, station) float32 20kB 298.8 ... 2.154e+03
    A Cr31/WinterPotato_    (time, station) float32 20kB 0.0 0.0 0.0 ... 0.0 0.0
    A Cr32/Sprinkler (ha    (time, station) float32 20kB 0.0 0.0 0.0 ... 0.0 0.0
    A Cr33/Surface (ha)     (time, station) float32 20kB 0.0 0.0 0.0 ... 0.0 0.0
    A Cr34/ShortRice_PAD    (time, station) float32 20kB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    header:   Advanced irrigation nodes               Demand and allocation r...
    scu:      86400
    t0:       2020-10-01 00:00:00

In [5]:
variable_names = list(ds.data_vars)

# Filter the list to only include variable names starting with 'P'
filtered_variable_names = [name for name in variable_names if name.startswith('P')]

In [6]:
def dt_to_str(df):
    return df.strftime('%m-%d')

def populate_crop_table(his_file_name, csv_file_name):
    df_dict = {
        'crop_id': [],
        'crop_name': [],
        'start_ts': [],
        'end_ts': [],
        'area_name': []
    }
    
    his_file = data_reader.HisFile(his_file_name)
    his_file.read(hia=True)
    ds = his_file.ds

    variable_names = list(ds.data_vars)
    potential_crop_names = [name for name in variable_names if name.startswith('P')]
    for potential_crop_name in potential_crop_names:
        crop_id_and_name = potential_crop_name.partition('P ')[2]
        crop_id = crop_id_and_name.split('/')[0]
        crop_name = crop_id_and_name.split('/')[1]
        
        for station in ds.station.values:
            ds_filtered = ds.sel({'station': station})[potential_crop_name]
            
            growing_times = ds_filtered.where(ds_filtered == 0, drop=True).time
            if growing_times.shape == (0,):
                start_of_year = str(ds_filtered.time.values.min())
            else:
                start_of_year = str(growing_times.values.min())
            year = int(start_of_year[:4])
            next_year = str(year + 1)
            end_of_year = next_year + start_of_year[4: ]

            if station == '5 / Blk_Air_20' and potential_crop_name == 'P Cr06/Orchard_CITRU':
                 5 == 5

            ds_filtered = ds_filtered.sel({'time': slice(start_of_year, end_of_year)})
            
            area_name_full = station.split(' / ')[1]
            area_name = area_name_full.split('_')[0]
                
            if ds_filtered.values.max() == 0:
                season_starts = [np.nan]
                season_ends = [np.nan]
            else:
                start = None
                season_starts = []
                season_ends = []
                for i, data in enumerate(ds_filtered):
                    if data.values != 0 and start is None:
                        start = i
                        season_starts.append(data.time.values)
                    if data.values == 0 and start is not None:
                        start = None
                        season_ends.append(ds_filtered.time.values[i - 1])
                if len(season_starts) > len(season_ends):
                    season_ends = copy.deepcopy(season_starts)

                for start_ts, end_ts in zip(season_starts, season_ends):
                    df_dict['crop_id'].append(crop_id)
                    df_dict['crop_name'].append(crop_name)
                    df_dict['start_ts'].append(start_ts)
                    df_dict['end_ts'].append(end_ts)
                    df_dict['area_name'].append(area_name)

    df = pd.DataFrame(df_dict)
    df = df.groupby(['crop_id', 'crop_name']).agg({'start_ts': 'min', 'end_ts': 'max'}).reset_index()
    df['start_ts'] = df['start_ts'].apply(dt_to_str)
    df['end_ts'] = df['end_ts'].apply(dt_to_str)
    return df

def populate_area_table(his_file_name, csv_file_name):
    df_dict = {
        'area_id': [],
        'area_name': [],
        'area_map_name': []
    }

    his_file = data_reader.HisFile(his_file_name)
    his_file.read()
    ds = his_file.ds

    for station in ds.station.values:
        area_id = station.split(' / ')[0]
        area_name_full = station.split(' / ')[1]
        area_name = area_name_full.split('_')[0]  

        df_dict['area_id'].append(area_id)
        df_dict['area_name'].append(area_name)
        df_dict['area_map_name'].append(None)

    df = pd.DataFrame(df_dict)
    return df

In [7]:
# df_crop = populate_crop_table(ribasim_dir / 'RIB_ADVIR_dmnd.his', None)
df_crop = populate_crop_table(src_dir / 'AirAgDmd.his', None)
df_crop_id = pd.DataFrame({'crop_name': df_crop['crop_name'].unique(), 'crop_name_fao': [None for i in range(len(df_crop['crop_name'].unique()))]})
# df_area = populate_area_table(ribasim_dir / 'RIB_ADVIR_dmnd.his', None)

In [33]:
ds.sel({'station': '5 / Blk_Air_20'})['P Cr06/Orchard_CITRU'].where(ds.sel({'station': '5 / Blk_Air_20'})['P Cr06/Orchard_CITRU'] == 0, drop=True).time.shape == (0,)

True

In [8]:
excel_path = src_dir / 'input_test_egypt.xlsx'
excel_path = '/Users/hemert/OneDrive - Stichting Deltares/Documents - International Delta Toolset/Salinity/Egypt/input.xlsx'
def write_excel_file(df, excel_path, sheet_name, append=False):
    # Open Excel file
    try:
        # book = load_workbook(excel_path)
        with pd.ExcelWriter(excel_path, engine="openpyxl", mode='a', if_sheet_exists='replace') as writer:
        # excel_file.book = book
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    except Exception as e:
        print(e)
        df.to_excel(excel_path, sheet_name=sheet_name, index=False)

In [9]:
write_excel_file(df_crop_id, excel_path, 'crop_id')
write_excel_file(df_crop, excel_path, 'crop')
# write_excel_file(df_area, excel_path, 'area')